Question 1 What is the cardinality in “basket data.csv”?

In [3]:
!wget -nc https://course.ccs.neu.edu/cs6220/spring2024/homeworks/01/basket_data.csv

--2024-01-17 08:34:04--  https://course.ccs.neu.edu/cs6220/spring2024/homeworks/01/basket_data.csv
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2357 (2.3K) [text/csv]
Saving to: ‘basket_data.csv’

basket_data.csv     100%[===================>]   2.30K  --.-KB/s    in 0s      

2024-01-17 08:34:04 (126 MB/s) - ‘basket_data.csv’ saved [2357/2357]



In [1]:
def cardinality_items(filename):
    """
    Takes a filename "*.csv" and returns an integer representing the cardinality of the set of all grocery items.
    """
    cardinality = set()
    with open(filename, 'r') as file:
        for line in file:
            items = line.strip().split(',')
            cardinality.update([item.strip() for item in items])
    return len(cardinality)


In [4]:
file_path = 'basket_data.csv'
cardinality_num = cardinality_items(file_path)
print(cardinality_num)

21


Q2 all_itemsets

In [5]:
def all_itemsets(items, k):
    """
    Generate all unique itemsets of size k from the given list of items.

    :param items: List of unique items.
    :param k: Size of each itemset.
    :return: List of all possible unique itemsets of size k.
    """
    if k == 0:
        return [[]]
    if len(items) < k:
        return []

    result = []
    for i in range(len(items)):
        # Generate all combinations that include items[i] and have size k
        for subset in all_itemsets(items[i+1:], k-1):
            result.append([items[i]] + subset)

    return result

In [6]:
example_items = ["ham", "cheese", "bread"]
k = 2
print("Generating itemsets of size", k, "from", example_items)
print(all_itemsets(example_items, k))

Generating itemsets of size 2 from ['ham', 'cheese', 'bread']
[['ham', 'cheese'], ['ham', 'bread'], ['cheese', 'bread']]


Q3 Netflix

In [1]:
#@title Download the data from website
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/movie_titles.csv
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_1.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_2.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_3.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_4.txt

from IPython.display import clear_output
clear_output()

print("Data in combined_data_1.txt looks like this: \n")
!head -5 combined_data_1.txt

print("\n\nData in movie_titles.csv looks like this: \n")
!head -5 movie_titles.csv

Data in combined_data_1.txt looks like this: 

1:
1488844,3,2005-09-06
822109,5,2005-05-13
885013,4,2005-10-19
30878,4,2005-12-26


Data in movie_titles.csv looks like this: 

1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW


In [2]:
f = open("movie_titles.csv", encoding ="cp1252")
data_lines = open("combined_data_1.txt", "r").readlines()

In [3]:
import os

def process_combined_data(file_paths):
    total_ratings = 0
    unique_users = set()
    years = set()
    user_ratings_count = {}

    for file_path in file_paths:
        with open(file_path, 'r') as file:
            for line in file:
                if ',' in line:
                    user_id, rating, date = line.strip().split(',')
                    total_ratings += 1
                    unique_users.add(user_id)
                    years.add(date.split('-')[0])
                    user_ratings_count[user_id] = user_ratings_count.get(user_id, 0) + 1
    return total_ratings, unique_users, years, user_ratings_count

def process_movie_titles(file_path):
    movie_names = {}
    with open(file_path, 'r', encoding='cp1252') as file:
        for line in file:
            _, _, movie_name = line.strip().split(',', 2)
            movie_names[movie_name] = movie_names.get(movie_name, 0) + 1
    unique_movie_names = len(set(movie_names.keys()))
    movies_with_four_names = sum(1 for count in movie_names.values() if count == 4)
    return unique_movie_names, movies_with_four_names

# Process combined_data_*.txt
combined_files = ["combined_data_1.txt", "combined_data_2.txt", "combined_data_3.txt", "combined_data_4.txt"]
total_ratings, unique_users, years, user_ratings_count = process_combined_data(combined_files)

# Process movie_titles.csv
movie_titles_file = "movie_titles.csv"
unique_movie_names, movies_with_four_names = process_movie_titles(movie_titles_file)

# Find users who rated exactly 200 movies
users_rated_200 = {user for user, count in user_ratings_count.items() if count == 200}

# Find the lowest user ID among them
lowest_user_id = min(users_rated_200)

print("3A Total movie rating records:", total_ratings)
print("3B Total unique users:", len(unique_users))
print("3C Year range:", min(years), "to", max(years))
print("4A Unique movie names:", unique_movie_names)
print("4B Movies with four different names:", movies_with_four_names)
print("5A number of users rated exactly 200 movies:", len(users_rated_200))

print("5B Lowest user ID who rated exactly 200 movies:", lowest_user_id)


3A Total movie rating records: 100480507
3B Total unique users: 480189
3C Year range: 1999 to 2005
4A Unique movie names: 17359
4B Movies with four different names: 5
5A number of users rated exactly 200 movies: 605
5B Lowest user ID who rated exactly 200 movies: 1001192


In [4]:
def find_user_5_star_ratings(combined_files, user_id):
    """
    Find all 5-star ratings given by a specific user.

    :param combined_files: List of paths to combined_data_*.txt files.
    :param user_id: The user ID whose 5-star ratings are to be found.
    :return: A set of movie IDs for which the user has given a 5-star rating.
    """
    favorite_movie_ids = set()
    for file_path in combined_files:
        with open(file_path, 'r') as file:
            current_movie_id = None
            for line in file:
                if ':' in line:
                    current_movie_id = line.split(':')[0].strip()
                else:
                    uid, rating, _ = line.strip().split(',')
                    if uid == user_id and rating == '5':
                        favorite_movie_ids.add(current_movie_id)
    return favorite_movie_ids



In [5]:
def get_movie_names(movie_ids, movie_titles_file):
    """
    Map movie IDs to their titles.

    :param movie_ids: A set of movie IDs.
    :param movie_titles_file: Path to the movie_titles.csv file.
    :return: A list of movie titles corresponding to the given movie IDs.
    """
    movie_names = []
    with open(movie_titles_file, 'r', encoding='cp1252') as file:
        for line in file:
            parts = line.strip().split(',', 2)
            if parts[0] in movie_ids:
                movie_names.append(parts[2])  # The movie title is the third element
    return movie_names


In [6]:
# Find all 5-star ratings by the lowest user ID
favorite_movie_ids = find_user_5_star_ratings(combined_files, lowest_user_id)

# Get the names of these movies
favorite_movie_names = get_movie_names(favorite_movie_ids, movie_titles_file)

print(f"5B Favorite movies of user {lowest_user_id}:")
for movie in favorite_movie_names:
    print(movie)


5B Favorite movies of user 1001192:
Sex and the City: Season 4
Ghost
Steel Magnolias
Pure Country
Finding Nemo (Full-screen)
